In [0]:
from pyspark.sql.types import *

In [0]:
%run "./utils/authenticate"

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1702461948814),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1702461948814),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1702461948814)]

In [0]:
df_geo_kinesis = (
    spark
    .readStream
    .format('kinesis')
    .option('streamName','streaming-0e36c8cd403d-geo')
    .option('initialPosition','earliest')
    .option('region','us-east-1')
    .option('awsAccessKey', ACCESS_KEY)
    .option('awsSecretKey', SECRET_KEY)
    .load()
)

df_pin_kinesis = (
    spark
    .readStream
    .format('kinesis')
    .option('streamName','streaming-0e36c8cd403d-pin')
    .option('initialPosition','earliest')
    .option('region','us-east-1')
    .option('awsAccessKey', ACCESS_KEY)
    .option('awsSecretKey', SECRET_KEY)
    .load()
)

df_user_kinesis = (
    spark
    .readStream
    .format('kinesis')
    .option('streamName','streaming-0e36c8cd403d-user')
    .option('initialPosition','earliest')
    .option('region','us-east-1')
    .option('awsAccessKey', ACCESS_KEY)
    .option('awsSecretKey', SECRET_KEY)
    .load()
)

In [0]:
%run "./utils/create_schemas"

In [0]:
df_user_pre_cleaning = (
    df_user_kinesis
    .selectExpr("CAST(data as STRING)")
    .select(F.from_json("data", user_schema).alias("user"))
    .select("user.*")
    )

df_geo_pre_cleaning = (
    df_geo_kinesis
    .selectExpr("CAST(data as STRING)")
    .select(F.from_json("data", geo_schema).alias("geo"))
    .select("geo.*")
)

df_pin_pre_cleaning = (
    df_pin_kinesis
    .selectExpr("CAST(data as STRING)")
    .select(F.from_json("data", pin_schema).alias("pin"))
    .select("pin.*")
)

In [0]:
%run "./utils/clean"

In [0]:
# display(df_pin)
# display(df_geo)
# display(df_user)

In [0]:
dbutils.fs.rm("/tmp/kinesis/_checkpoints/", True)
(
    df_pin.writeStream
    .format("delta") 
    .outputMode("append") 
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") 
    .table("0e36c8cd403d_pin_table")
)

Out[14]: <pyspark.sql.streaming.StreamingQuery at 0x7f2f8ed4b6a0>

In [0]:
dbutils.fs.rm("/tmp/kinesis/_checkpoints/", True)
(
    df_geo.writeStream 
    .format("delta") 
    .outputMode("append") 
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") 
    .table("0e36c8cd403d_geo_table")
)

Out[15]: <pyspark.sql.streaming.StreamingQuery at 0x7f2f8ed4bc70>

In [0]:
dbutils.fs.rm("/tmp/kinesis/_checkpoints/", True)
(
    df_user.writeStream 
    .format("delta") 
    .outputMode("append") 
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") 
    .table("0e36c8cd403d_user_table")
)

Out[16]: <pyspark.sql.streaming.StreamingQuery at 0x7f2f8ed4bd90>